# Placeholder for importing data into DiveDB

## Example importing EDF+CSV data

## Example importing NetCDF data

In [ ]:
import xarray as xr
#import netcdf4

# Load the NetCDF file
file_path = f"{data_dir}/2004001_TrackTDR_RawCurated.nc"
dataset = xr.open_dataset(file_path)

# Print the dataset information
print(dataset)